In [1]:
import os, json, glob, numpy as np, pandas as pd
from pandas import HDFStore
pd.set_option("display.max_columns", 200)
from google.colab import drive
if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
OUT = "/content/drive/MyDrive/Data/VAEP_outputs"
H5  = "/content/drive/MyDrive/Data/Processed/worldcup2022_spadl_merged.h5"

ACTIONS_ALL = os.path.join(OUT, "wc2022_all_COMPOSED_vaep_actions.csv")
ACTIONS_ON  = os.path.join(OUT, "wc2022_ononly_COMPOSED_vaep_actions.csv")
ACTIONS_NO  = os.path.join(OUT, "wc2022_no_onoff_COMPOSED_vaep_actions.csv")

def build_name_map_from_h5(h5_path) -> pd.DataFrame:
    rows = []
    with HDFStore(h5_path, mode="r") as store:
        for k in store.keys():
            if not k.startswith("/actions/"):
                continue
            df = store[k]
            need = [c for c in ["event_uuid","game_id","team_id","team_name","player_id","player_name"] if c in df.columns]
            if "event_uuid" in need and "game_id" in need:
                rows.append(df[need].copy())
    if not rows:
        raise RuntimeError("No finding column in h5")
    M = pd.concat(rows, axis=0, ignore_index=True).drop_duplicates(subset=["event_uuid"])
    for c in ["game_id","team_id","player_id"]:
        if c in M.columns:
            M[c] = pd.to_numeric(M[c], errors="coerce")
    return M

def player_master_from_h5(h5_path) -> pd.DataFrame:
    rows = []
    with HDFStore(h5_path, mode="r") as store:
        for k in store.keys():
            if not k.startswith("/actions/"):
                continue
            df = store[k]
            if {"player_id","player_name"}.issubset(df.columns):
                rows.append(df[["player_id","player_name"]])
    if not rows:
        return pd.DataFrame(columns=["player_id","player_name"])
    P = pd.concat(rows, axis=0, ignore_index=True).dropna()
    P = (P.groupby("player_id")["player_name"]
           .agg(lambda s: s.value_counts().idxmax())
           .reset_index())
    P["player_id"] = pd.to_numeric(P["player_id"], errors="coerce")
    return P.dropna(subset=["player_id"])

def team_master_from_h5(h5_path) -> pd.DataFrame:
    rows = []
    with HDFStore(h5_path, mode="r") as store:
        for k in store.keys():
            if not k.startswith("/actions/"):
                continue
            df = store[k]
            if {"team_id","team_name"}.issubset(df.columns):
                rows.append(df[["team_id","team_name"]])
    if not rows:
        return pd.DataFrame(columns=["team_id","team_name"])
    T = pd.concat(rows, axis=0, ignore_index=True).dropna()
    T = (T.groupby("team_id")["team_name"]
           .agg(lambda s: s.value_counts().idxmax())
           .reset_index())
    T["team_id"] = pd.to_numeric(T["team_id"], errors="coerce")
    return T.dropna(subset=["team_id"])

NAME_MAP = build_name_map_from_h5(H5)
P_MASTER = player_master_from_h5(H5)
T_MASTER = team_master_from_h5(H5)

def load_actions_with_names(csv_path, augment=True):
    A = pd.read_csv(csv_path)
    A = A.merge(NAME_MAP, on="event_uuid", how="left", suffixes=("","_h5"))
    if augment:
        if "player_id" in A.columns and not P_MASTER.empty:
            A = A.merge(P_MASTER, on="player_id", how="left", suffixes=("","_by_id"))
            if "player_name" in A.columns and "player_name_by_id" in A.columns:
                A["player_name"] = A["player_name"].fillna(A["player_name_by_id"])
                A.drop(columns=["player_name_by_id"], inplace=True)
        if "team_id" in A.columns and not T_MASTER.empty:
            A = A.merge(T_MASTER, on="team_id", how="left", suffixes=("","_by_id"))
            if "team_name" in A.columns and "team_name_by_id" in A.columns:
                A["team_name"] = A["team_name"].fillna(A["team_name_by_id"])
                A.drop(columns=["team_name_by_id"], inplace=True)
    return A


# 1) Semi-final team VAEP per match
semi_names = ["Argentina", "Croatia", "France", "Morocco"]

tm = T_MASTER.copy()
tm["team_name_lc"] = tm["team_name"].astype(str).str.lower()
want = [s.lower() for s in semi_names]

semi_ids = tm[tm["team_name_lc"].isin(want)]["team_id"].unique()
if len(semi_ids) != 4:
    semi_ids = tm[tm["team_name_lc"].str.contains("|".join(want), na=False)]["team_id"].unique()
print("[INFO] semi-final team_ids:", semi_ids)

A_all = load_actions_with_names(ACTIONS_ALL, augment=True)

team_game_id = (
    A_all.groupby(["team_id","game_id"], dropna=False)[
        ["offensive_value","defensive_value","vaep_value"]
    ]
    .sum()
    .reset_index()
)

semi_team_games = team_game_id[team_game_id["team_id"].isin(semi_ids)].copy()

matches = semi_team_games.groupby("team_id").size().reset_index(name="matches")
avg_vals = (
    semi_team_games.groupby("team_id")[["offensive_value","defensive_value","vaep_value"]]
    .mean()
    .reset_index()
)

semi_avg = (
    avg_vals.merge(matches, on="team_id", how="left")
            .merge(T_MASTER[["team_id","team_name"]].drop_duplicates("team_id"),
                   on="team_id", how="left")
            [["team_id","team_name","matches","offensive_value","defensive_value","vaep_value"]]
            .sort_values("vaep_value", ascending=False)
)

print("=== Semi-finalists: mean per match (ALL/COMPOSED, STRICT) ===")
display(semi_avg)

# 2) Top5 ON-ONLY / NO-ONOFF

def top5_players_per_match(actions_csv, title):
    A = load_actions_with_names(actions_csv, augment=True)
    pg = (A.groupby(["player_id","player_name","team_id","game_id"], dropna=False)["vaep_value"]
            .sum().reset_index(name="vaep_game"))
    P = (pg.groupby(["player_id","player_name","team_id"], dropna=False)["vaep_game"]
            .agg(games="count", total_vaep="sum", avg_vaep_per_match="mean")
            .reset_index()
            .sort_values("avg_vaep_per_match", ascending=False)
            .head(5))
    print(f"=== Top5 players by avg VAEP per match — {title} (STRICT) ===")
    display(P[["player_id","player_name","team_id","games","total_vaep","avg_vaep_per_match"]])

top5_players_per_match(ACTIONS_ON, "ON-ONLY")
top5_players_per_match(ACTIONS_NO, "NO-ONOFF")


# 3) ON-ONLY Top5: avg VAEP per match  vs  avg xG(shot_xg) per match
A_on = load_actions_with_names(ACTIONS_ON, augment=True)
pg_on = (A_on.groupby(["player_id","player_name","game_id"], dropna=False)["vaep_value"]
             .sum().reset_index(name="vaep_game"))
P_on  = (pg_on.groupby(["player_id","player_name"], dropna=False)["vaep_game"]
             .agg(games="count", avg_vaep_per_match="mean")
             .reset_index())
Top5_on = P_on.sort_values("avg_vaep_per_match", ascending=False).head(5)

rows = []
with HDFStore(H5, mode="r") as store:
    for k in store.keys():
        if not k.startswith("/actions/"):
            continue
        df = store[k]
        if {"player_id","player_name","game_id","shot_xg"}.issubset(df.columns):
            rows.append(df[["player_id","player_name","game_id","shot_xg"]].copy())
if not rows:
    raise FileNotFoundError("No 'shot_xg' in h5")

XG = pd.concat(rows, axis=0, ignore_index=True)
xg_game = (XG.groupby(["player_id","player_name","game_id"], dropna=False)["shot_xg"]
             .sum().reset_index(name="xg_game"))

vaep_games = pg_on[["player_id","player_name","game_id"]].drop_duplicates()
vx = vaep_games.merge(xg_game, on=["player_id","player_name","game_id"], how="left").fillna({"xg_game":0.0})
xg_avg = (vx.groupby(["player_id","player_name"], dropna=False)["xg_game"]
            .mean().reset_index(name="xg_per_match"))

cmp = Top5_on.merge(xg_avg, on=["player_id","player_name"], how="left") \
             .rename(columns={"avg_vaep_per_match":"ononly_vaep_per_match"})

print("=== ON-ONLY Top5: avg VAEP per match vs avg xG per match (STRICT) ===")
display(cmp[["player_id","player_name","ononly_vaep_per_match","xg_per_match"]]
        .sort_values("ononly_vaep_per_match", ascending=False))

[INFO] semi-final team_ids: [771 779 785 788]
=== Semi-finalists: mean per match (ALL/COMPOSED, STRICT) ===


,team_id,team_name,matches,offensive_value,defensive_value,vaep_value
1,779,Argentina,7,0.357463,0.085957,0.443421
0,771,France,7,0.053256,0.038041,0.091297
2,785,Croatia,7,-0.078097,-0.002386,-0.080482
3,788,Morocco,7,-0.482456,-0.202057,-0.684513


=== Top5 players by avg VAEP per match — ON-ONLY (STRICT) ===


,player_id,player_name,team_id,games,total_vaep,avg_vaep_per_match
203,5571.0,Hirving Rodrigo Lozano Bahena,794,3,1.351161,0.450387
624,39565.0,Jamal Musiala,770,3,1.240161,0.413387
329,8400.0,Serge Gnabry,770,3,1.180644,0.393548
17,3089.0,Kevin De Bruyne,782,3,1.134591,0.378197
323,8246.0,Christian Pulisic,1839,4,1.511076,0.377769


=== Top5 players by avg VAEP per match — NO-ONOFF (STRICT) ===


,player_id,player_name,team_id,games,total_vaep,avg_vaep_per_match
203,5571.0,Hirving Rodrigo Lozano Bahena,794,3,1.283979,0.427993
624,39565.0,Jamal Musiala,770,3,1.238566,0.412855
329,8400.0,Serge Gnabry,770,3,1.212804,0.404268
184,5503.0,Lionel Andrés Messi Cuccittini,779,7,2.644505,0.377786
214,5591.0,Filip Kostić,786,2,0.742067,0.371033


=== ON-ONLY Top5: avg VAEP per match vs avg xG per match (STRICT) ===


,player_id,player_name,ononly_vaep_per_match,xg_per_match
0,5571.0,Hirving Rodrigo Lozano Bahena,0.450387,0.080135
1,39565.0,Jamal Musiala,0.413387,0.356523
2,8400.0,Serge Gnabry,0.393548,0.242673
3,3089.0,Kevin De Bruyne,0.378197,0.059322
4,8246.0,Christian Pulisic,0.377769,0.194471
